In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from multithreaded_computation_simple import *

In [3]:
c = Config()

In [4]:
c.NUM_PROC = 2
c.NUM_THREADS = 3
c.MIN_INST_PER_THREAD = 3
c.MAX_INST_PER_THREAD = 3

cs_ranges = [1, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]
# cs_ranges = [1000,]

o = MySolver()
dag = DAG(c, o)
dag.add_constraints(cs_delay=False)
ws = WorkStealingAlgorithm(dag)
ws.add_constraints()
opt = OptimalAlgorithm(dag)
opt.add_constraints()

low = 1
high = 4

ratios = []
for r in cs_ranges:
    print(f"Trying with {r}")
    o.push()
    for T in range(c.NUM_THREADS):
        for i in range(c.MAX_INST_PER_THREAD):
            o.add(dag.cs_delay_per_thread[T][i] == r)
    # dag.cs_delay_constraints(r)
    ans, m = binary_search(o, ws.time_taken, opt.time_taken, low, high)
    o.pop()
    print(f"{r} --> {ans}")
    ratios.append(ans)
    low = int(ans)

ratios

Origin Processor 1 sat
Origin Processor 2 sat
Stolen sat
Work Stealing sat
Work Conserving sat
Trying with 1
Checking for 2.5


In [9]:
import plotly.express as px
import pandas as pd

df = pd.DataFrame(dict(
    x = cs_ranges,
    y = ratios
))

fig = px.line(df, x="x", y="y",
        labels={
                "x": 'Context Shift Delay Ratio',
                "y": "Ratio",
                })
fig.update_layout(autosize=False)
fig.write_html("plot2.html")
fig

In [ ]:
for v in o.variables:
    print(v,"=", m[o.variables[v]])

In [ ]:
delay_to_ratio[c.CONTEXT_SHIFT_DELAY_MIN] = ans

In [ ]:
CS_k = [0, 0.1, 0.3, 0.7, 1, 5, 10, 20, 50, 100]
P2T3I2 = [1.5, 1.53125, 1.592, 1.65, 1.71, 2.14, 2.28, 2.38, 2.44, 2.47]
P3T3I2 = [1, 1.0625, 1.18, 1.375, 1.49, 1.828, 1.85, 1.9, 1.96, 1.97]
# P2T3I3 = [1.5, 0, 0, 0, 0, 0 ,0 ,0 , 2.875+, 2.9375+]

unequal_P2T3I2 = [1.5, 1.53125 to <1.546875]
# 0.1 -- 1.535?
# 100 -- 68
# 10 -- 8
# 1 -- 2
# 0.3 -- 1.6+ to 1.6125
# 0.5 -- 1.664